In [14]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)
pd.options.display.max_colwidth=1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
nrows = 100000
train = prepare_data('train', nrows=nrows, convert_action_type=False, add_test=False, recompute=False)
# df = compute_session_fts(train, 'train')

[05-27 20:41:48 - create_model_inputs-110 - prepare_data - INFO] Load from existing ./nn_cache/train_100000_no_test.snappy


In [4]:
train_ids = np.load('./nn_cache/train_ids.npy')

In [5]:
len(train_ids)

9234

In [6]:
ids_to_test = ['65a684aa75ea1']

In [55]:
# choose an id to look at
rid = np.random.choice(train_ids, 1)[0]
rid

'0d29f6d11363f'

In [56]:
train[train.session_id==rid]

,session_id,timestamp,action_type,current_filters,reference,impressions,prices
1724,0d29f6d11363f,2018-11-05 11:17:21,clickout item,None,3756812,3756812|25955|36450|147276|3974716|2282300|151827|915997|107368|1258671|6340020|80225|9374742|2805597|1535855|4572856|36453|1392892|4440056|7195548|4894618|6533954|2831136|1773777|935753,16|102|101|70|86|66|21|16|12|85|64|63|21|55|72|47|28|137|65|13|72|28|26|36|25
1725,0d29f6d11363f,2018-11-05 11:29:27,search for destination,None,"Ankara, Turkey",None,None
1726,0d29f6d11363f,2018-11-05 11:30:15,clickout item,None,2610238,151827|107368|1510073|5081140|36453|2162478|6533954|2831136|2610238|1375262|1773777|6967974|36470|6365508|6317924|3484880|36454|1625365|3163544|5746558|1364394|2312954|986837|5798414|1489021,21|12|21|27|28|25|28|26|19|30|36|24|36|25|27|32|27|23|32|33|31|32|25|24|18
1727,0d29f6d11363f,2018-11-05 11:32:41,clickout item,None,1489021,151827|107368|1510073|5081140|36453|2162478|6533954|2831136|2610238|1375262|1773777|6967974|36470|6365508|6317924|3484880|36454|1625365|3163544|5746558|1364394|2312954|986837|5798414|1489021,21|12|21|27|28|25|28|26|19|30|36|24|36|25|27|32|27|23|32|33|31|32|25|24|18
1728,0d29f6d11363f,2018-11-05 11:35:52,clickout item,None,2816552,107368|3756812|915997|2816552|1099020|5166254|7195548|2246700|2552126|107363|1178038|1967767|2168390|6785478|2330108|107365|107361|9993096|10064270|4560832|10755202|6965742,12|16|16|14|17|8|13|16|15|17|14|14|7|13|16|13|14|14|15|10|17|15
1729,0d29f6d11363f,2018-11-05 11:38:24,clickout item,None,3756812,107368|3756812|915997|2816552|1099020|5166254|7195548|2246700|2552126|107363|1178038|1967767|2168390|6785478|2330108|107365|107361|9993096|10064270|4560832|10755202|6965742,12|16|16|14|17|8|13|16|15|17|14|14|7|13|16|13|14|14|15|10|17|15
1730,0d29f6d11363f,2018-11-05 11:40:12,clickout item,None,3756812,107368|3756812|915997|2816552|1099020|5166254|7195548|2246700|2552126|107363|1178038|1967767|2168390|6785478|2330108|107365|107361|9993096|10064270|4560832|10755202|6965742,12|16|16|14|17|8|13|16|15|17|14|14|7|13|16|13|14|14|15|10|17|15
1731,0d29f6d11363f,2018-11-05 11:41:14,clickout item,None,1178038,107368|3756812|915997|2816552|1099020|5166254|7195548|2246700|2552126|107363|1178038|1967767|2168390|6785478|2330108|107365|107361|9993096|10064270|4560832|10755202|6965742,12|16|16|14|17|8|13|16|15|17|14|14|7|13|16|13|14|14|15|10|17|15
1732,0d29f6d11363f,2018-11-05 11:41:45,clickout item,None,1178038,107368|3756812|915997|2816552|1099020|5166254|7195548|2246700|2552126|107363|1178038|1967767|2168390|6785478|2330108|107365|107361|9993096|10064270|4560832|10755202|6965742,12|16|16|14|17|8|13|16|15|17|14|14|7|13|16|13|14|14|15|10|17|15
1733,0d29f6d11363f,2018-11-05 11:46:32,interaction item image,None,2168390,None,None


In [57]:
mask = train_ids == rid

### history

In [51]:
train_hist = np.load('./nn_cache/train_history.npy')#[:100]

In [52]:
train_hist.shape

(9234, 25, 4)

In [58]:
train_hist[mask].shape

(13, 25, 4)

In [64]:
train_hist[mask][-1]

array([[0.0, 2.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 1.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 3.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0]], dtype=object)

### impressions

In [65]:
train_imp = np.load('./nn_cache/train_impression.npy')

In [66]:
train_imp[mask].shape

(13, 25, 157)

In [77]:
train_imp[mask][-1]

array([[0, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [68]:
from create_model_inputs import meta_encoding

In [74]:
m, _ = meta_encoding()

[05-27 21:26:07 - create_model_inputs-47 - meta_encoding - INFO] Load from existing file: ./nn_cache/meta_mapping.npy


In [79]:
m[1773777]

array([0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0])

### prices

In [80]:
prices = np.load('./nn_cache/train_price.npy')

In [81]:
prices[mask][0]

array([[2.83321334, 0.14285714],
       [4.63472899, 0.95238095],
       [4.62497281, 0.9047619 ],
       [4.26267988, 0.71428571],
       [4.46590812, 0.85714286],
       [4.20469262, 0.66666667],
       [3.09104245, 0.19047619],
       [2.83321334, 0.14285714],
       [2.56494936, 0.04761905],
       [4.4543473 , 0.80952381],
       [4.17438727, 0.57142857],
       [4.15888308, 0.52380952],
       [3.09104245, 0.19047619],
       [4.02535169, 0.47619048],
       [4.29045944, 0.76190476],
       [3.87120101, 0.42857143],
       [3.36729583, 0.33333333],
       [4.92725369, 1.        ],
       [4.18965474, 0.61904762],
       [2.63905733, 0.0952381 ],
       [4.29045944, 0.76190476],
       [3.36729583, 0.33333333],
       [3.29583687, 0.28571429],
       [3.61091791, 0.38095238],
       [3.25809654, 0.23809524]])

In [ ]:
16|102|101|70|86|66|21|16|12|85|64|63|21|55|72|47|28|137|65|13|72|28|26|36|25

In [82]:
np.log1p(16)

2.833213344056216

### take a look of test sessions that are not in submission

In [ ]:
test = load_data('test')

In [ ]:
test_sub = load_data('submission_popular')

In [ ]:
test_sub.shape

In [ ]:
f'{test_sub.session_id.nunique():,}'

In [ ]:
sub_ids = test_sub.session_id.unique()

In [ ]:
f'{test.shape[0]:,}'

In [ ]:
subs = test[test.session_id.isin(sub_ids)]

In [ ]:
f'{subs.shape[0]:,}'

In [ ]:
no_subs = test[~test.session_id.isin(sub_ids)]

In [ ]:
f'{no_subs.shape[0]:,}'

In [ ]:
no_subs_ids = no_subs.session_id.unique()

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
no_subs[no_subs.session_id==np.random.choice(no_subs_ids, 1)[0]]